In [ ]:
% load_ext autoreload
% autoreload 2

In [ ]:
import numpy as np
import lightkurve as lk
import threemultis

In [ ]:
tpf = lk.search_targetpixelfile('K2-168').download()
tpf = tpf[np.in1d(tpf.time, tpf.to_lightcurve(aperture_mask='all').remove_nans().time)]
tpf = tpf[tpf.to_lightcurve().normalize().flux > 0.8]
mask = threemultis.utils.planet_mask(tpf.time, 'K2-168')

In [ ]:
aper = np.nanmedian(tpf.flux, axis=0) > 30

In [ ]:
# First pass, remove some very bad outliers
bad = np.zeros(len(tpf.time), bool)
for count in range(2):
    pld_lc = tpf[~bad].to_corrector('pld').correct(aperture_mask=aper, cadence_mask=mask[~bad])
    pld_lc = pld_lc.flatten(31, mask=~mask[~bad])
    bad |= np.in1d(tpf.time, pld_lc.time[np.abs(pld_lc.flux - 1) > 5 * np.std(pld_lc.flux - 1)])
    
tpf = tpf[~bad]
mask = mask[~bad]

In [ ]:
tpf.plot(scale='log', aperture_mask=aper)

In [ ]:
params = threemultis.utils.get_params('K2-168')

In [ ]:
clc = threemultis.jointfit.joint_fit(tpf, period_value=list(params['Period']),
                                            t0_value=list(params['T0']), depth_value=list(params['Depth']),
                                            duration_value=list(params['T0']),
                                            R_star=(params.loc[0, 'R_star'], params.loc[0, 'R_star_error']),
                                            M_star=(params.loc[0, 'M_star'], params.loc[0, 'M_star_error']),
                                            T_star=(params.loc[0, 'T_star'], params.loc[0, 'T_star_error']),
                                           aperture=aper, return_quick_corrected=True, trim=2)

In [ ]:
clc.plot()

In [ ]:
threemultis.utils.planet_plot(clc, 'K2-168');